# Long-term Conversation

**Long-term, open-domain conversations** over multiple sessions present significant challenges, as they require the system to retain past events and user preferences to deliver coherent and personalized responses. We explore the impact of different **memory granularities** and present two key findings:

- *Turn-level*, *session-level*, and *summarization-based* methods all exhibit limitations. 

- The redundancy in natural language introduces noise, hindering precise retrieval. *LLMLingua-2*, originally designed for prompt compression, can serve as an effective denoising method to enhance memory retrieval accuracy.

Building on these insights, we propose to construct the memory bank at segment level by introducing a conversation **Se**gmentation model that partitions long-term conversations into topically coherent segments, while applying **Com**pression based denoising on memory units to enhance memory retrieval.

In [1]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
import json

conversation_history = json.load(open("example.json", "r"))

requests = [
    'Implement a function to find the number of "lucky" ternary tree using the given definition.'
]

In [ ]:
from secom import SeCom

# For light-weight memory management without GPU, use fast config which uses BM25 retriever without compressor.
# See long-term_chat_cpu.ipynb for more details.
memory_manager_seg = SeCom(
    granularity="segment", config_path="../secom/configs/mpnet.yaml"
)
memory_seg = memory_manager_seg.get_memory(
    requests, conversation_history, retrieve_topk=1
)

print(memory_seg["retrieved_texts"][0])

0-th session is segmented to 8 segments
1-th session is segmented to 8 segments
2-th session is segmented to 4 segments
3-th session is segmented to 4 segments


Token indices sequence length is longer than the specified maximum sequence length for this model (773 > 512). Running this sequence through the model will result in indexing errors


4-th session is segmented to 7 segments
<Turn 0>: [human]: A binary tree is full if all of its vertices have either zero or two children. Let B_n denote the number of full binary trees with n vertices. Implement a function to find B_n.
[bot]: To find the number of full binary trees with \(n\) vertices, denoted as \(B_n\), we can use a dynamic programming approach or directly apply the formula derived from Catalan numbers, given that \(n\) must be odd for a full binary tree (since every node has either 0 or 2 children, and thus, for \(n\) nodes, there are \(n-1\) edges, leading to an even number of total nodes and edges, which means \(n\) itself must be odd).

The \(n\)th Catalan number is given by:
\[ C_n = \frac{1}{n+1}\binom{2n}{n} \]

Since a full binary tree with \(n\) vertices corresponds to the \((n-1)/2\)th Catalan number (because a full binary tree with \(n\) vertices has \((n-1)/2\) internal nodes, each having exactly 2 children), we can find \(B_n\) using the formula for Cata

In [9]:
memory_manager_turn = SeCom(
    granularity="turn", config_path="../secom/configs/mpnet.yaml"
)
memory_turn = memory_manager_turn.get_memory(
    requests, conversation_history, compress_rate=0.9, retrieve_topk=1
)
print(memory_turn["retrieved_texts"][0])

Token indices sequence length is longer than the specified maximum sequence length for this model (773 > 512). Running this sequence through the model will result in indexing errors


<Turn 1>: [human]: What if the problem changed from a binary tree to a ternary tree?
[bot]: If the problem changes from a binary tree to a ternary tree, the concept of full trees adjusts accordingly. A full ternary tree is one where every node has either zero or three children. This change impacts the calculation of the number of such trees with \(n\) vertices, denoted as \(T_n\) for ternary trees.

For ternary trees, the relationship between the number of vertices and the structure of the trees becomes more complex. Unlike binary trees, where we could directly relate the problem to Catalan numbers, the enumeration of full ternary trees doesn't directly map to a well-known sequence in the same straightforward manner. However, the problem can still be approached through combinatorial methods or dynamic programming.

The number of full ternary trees with \(n\) vertices does not have a simple closed-form formula like the Catalan numbers for binary trees. However, if one insists on a metho

## Start a conversation with SeCom

SeCom is versatile and can be adapted for various models and specific requirements in memory management. Users can specify different model names and configurations as needed for their particular use case.

In [2]:
import sys

sys.path.append("../secom")
from utils import OpenAILLM

llm = OpenAILLM()
prompt_template = """
Response to the request baesd on the conversation history. 

Conversation history:
{conversation_history}

Request:
{request}

"""

In [13]:
# using SeCom
relevant_history = memory_seg["retrieved_texts"][0]
prompt = prompt_template.format(
    conversation_history=relevant_history, request=requests[0]
)
response_seg = llm(prompt)
print(response_seg)

Given the definition of a "lucky" ternary tree where each vertex has either zero or one child, we can conclude that the structure remains similar to that of a "lucky" binary tree. In this case, a "lucky" ternary tree would also behave like a linked list, where each node has at most one child.

Therefore, for a "lucky" ternary tree with \(n\) vertices, the number of ways to construct such a tree is also exactly one, regardless of the number of vertices. This is because you can only add one child successively until you reach the total of \(n\) vertices. 

Thus, the number of "lucky" ternary trees \(T_n\) is:

\[ T_n = 1 \text{ for all } n \geq 1 \]

Here's a simple Python function to represent this:

```python
def T_n(n):
    if n < 1:
        return "n must be at least 1"
    return 1

# Example usage
n = 5
print(f"T_{n}:", T_n(n))
```

This function checks if \(n\) is at least 1 and returns 1, reflecting the fact that there is exactly one way to construct a "lucky" ternary tree with \(

In [14]:
# using Turn-level memory management
relevant_history = memory_turn["retrieved_texts"][0]
prompt = prompt_template.format(
    conversation_history=relevant_history, request=requests[0]
)
response_turn = llm(prompt)
print(response_turn)

To implement a function that calculates the number of "lucky" ternary trees, we first need to establish what defines a "lucky" ternary tree based on the previous discussion. Since the definition wasn't provided, let's assume that a "lucky" ternary tree is a full ternary tree where every node has exactly three children, and we are allowed to have a specific number of vertices \( n \). 

To find the number of such trees for a given \( n \), we can use a dynamic programming approach based on the recursive structure of ternary trees. The key observation is that for a full ternary tree, if we have \( n \) vertices, the root will have three children, and the total number of vertices in the three subtrees combined must be \( n - 1 \) (since one vertex is the root). 

Let’s denote \( T(n) \) as the number of full ternary trees with \( n \) vertices. The recursive relationship can be established as follows:

1. If a tree has \( n \) vertices, the root takes one vertex, leaving \( n-1 \) vertice

## Incoperate the newly evolved interaction turn to SeCom

Users can use `update_memory()` to incoperate the newly evolved user-bot interaction turn into the memory bank to serve infinitely long conversation.

In [11]:
import json
from secom import SeCom

memory_manager = SeCom(granularity="segment", config_path="../secom/configs/mpnet.yaml")

conversation_history = json.load(open("example.json", "r"))
requests = [
    "What advancements in molecular biology are expected to occur in the next century?"
]

memory = memory_manager.get_memory(requests, conversation_history, retrieve_topk=1)

relevant_history = memory["retrieved_texts"][0]
prompt = prompt_template.format(
    conversation_history=relevant_history, request=requests[0]
)
response = llm(prompt)

# update memory bank
new_turn = f"[Human]: {requests[0]}\n\n[Bot]: {response}"
memory_manager.update_memory(new_turn)
print(memory_manager.memory_bank[-1])

requests = [
    'Implement a function to find the number of "lucky" ternary tree using the given definition.'
]
memory = memory_manager.get_memory(requests, retrieve_topk=1)

relevant_history = memory["retrieved_texts"][0]
prompt = prompt_template.format(
    conversation_history=relevant_history, request=requests[0]
)
response = llm(prompt)

# update memory bank
new_turn = f"[Human]: {requests[0]}\n\n[Bot]: {response}"
memory_manager.update_memory(new_turn)
print(memory_manager.memory_bank[-1])

0-th session is segmented to 8 segments
1-th session is segmented to 8 segments
2-th session is segmented to 4 segments
3-th session is segmented to 4 segments


Token indices sequence length is longer than the specified maximum sequence length for this model (773 > 512). Running this sequence through the model will result in indexing errors


4-th session is segmented to 8 segments
page_content='<Turn 14>:[human] : How you think about future of molecular biology in next century?
[bot] : future of molecular biology in next century holds immense promise and potential for groundbreaking discoveries and innovations. With rapid advancements in technology and increasing understanding of molecular mechanisms, we can expect significant progress in areas as personalized medicine, where treatments and drugs tailored to individual's genetic makeup., field of synthetic biology, involves redesigning organisms for specific purposes, could revolutionize industries from pharmaceuticals to biofuels. integration of artificial intelligence and machine learning with molecular biology could further accelerate research and development, leading to faster discoveries and applications in healthcare, agriculture, and environmental conservation. Overall, future of molecular biology is likely to be characterized by a deeper understanding of life at mo